# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [4]:

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy


In [14]:
df = pd.read_csv('previsao_de_renda.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.
    

# 1

In [18]:
df.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [20]:
#1
df = df.drop(columns=['Unnamed: 0','data_ref', 'id_cliente'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


In [49]:
#df.groupby(['qtd_filhos']).sum()

In [24]:
categoria_mais_frequente = df['qtd_filhos'].value_counts().idxmax()
categoria_mais_frequente

0

In [26]:
categoria_mais_frequente2 = df['qt_pessoas_residencia'].value_counts().idxmax()
categoria_mais_frequente2

2.0

In [28]:
categoria_mais_frequente3 = df['tipo_renda'].value_counts()
categoria_mais_frequente3

tipo_renda
Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: count, dtype: int64

In [30]:
df.columns

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [55]:
df1 = smf.ols(''' np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos, Treatment(7)) 
                + tipo_renda + educacao + idade + estado_civil + tipo_residencia + idade
                + tempo_emprego + qt_pessoas_residencia''', data = df).fit()
df1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     230.6
Date:                Fri, 26 Jul 2024   Prob (F-statistic):               0.00
Time:                        16:25:54   Log-Likelihood:                -13561.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12396   BIC:                         2.741e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             4.5282      1.026      4.414      0.000       2.517       6.539
sexo[T.M]                             0.7882      0.015     53.731      0.000       0.759       0.817
posse_de_veiculo[T.True]              0.0433      0.014      3.063      0.002       0.016       0.071
posse_de_imovel[T.True]               0.0830      0.014      5.927      0.000       0.056       0.110
C(qtd_filhos, Treatment(7))[T.0]      2.0322      0.820      2.477      0.013       0.424       3.640
C(qtd_filhos, Treatment(7))[T.1]      1.7792      0.723      2.461      0.014       0.362       3.196
C(qtd_filhos, Treatment(7))[T.2]      1.5184      0.629      2.415      0.016       0.286       2.751
C(qtd_filhos, Treatment(7))[T.3]      1.1080      0.541      2.047      0.041       0.047       2.169
C(qtd_filhos, Treatment(7))[T.4]      1.2144      0.489      2.482      0.013       0.255       2.173
C(qtd_filhos, Treatment(7))[T.5]      1.0028      0.643      1.560      0.119      -0.258       2.263
C(qtd_filhos, Treatment(7))[T.14]    -2.7621      0.967     -2.857      0.004      -4.657      -0.867
tipo_renda[T.Bolsista]                0.2242      0.241      0.930      0.353      -0.248       0.697
tipo_renda[T.Empresário]              0.1551      0.015     10.389      0.000       0.126       0.184
tipo_renda[T.Pensionista]            -0.3119      0.241     -1.293      0.196      -0.785       0.161
tipo_renda[T.Servidor público]        0.0556      0.022      2.501      0.012       0.012       0.099
educacao[T.Pós graduação]             0.0995      0.159      0.625      0.532      -0.212       0.411
educacao[T.Secundário]               -0.0139      0.072     -0.192      0.847      -0.155       0.127
educacao[T.Superior completo]         0.0932      0.072      1.290      0.197      -0.048       0.235
educacao[T.Superior incompleto]      -0.0423      0.078     -0.544      0.587      -0.195       0.110
estado_civil[T.Separado]              0.3311      0.111      2.976      0.003       0.113       0.549
estado_civil[T.Solteiro]              0.2739      0.109      2.515      0.012       0.060       0.487
estado_civil[T.União]                -0.0340      0.025     -1.351      0.177      -0.083       0.015
estado_civil[T.Viúvo]                 0.3735      0.116      3.231      0.001       0.147       0.600
tipo_residencia[T.Casa]              -0.0435      0.054     -0.804      0.421      -0.150       0.063
tipo_residencia[T.Com os pais]       -0.0245      0.060     -0.406      0.685      -0.143       0.094
tipo_residencia[T.Comunitário]       -0.1204      0.106     -1.139      0.255      -0.328       0.087
tipo_residencia[T.Estúdio]            0.0632      0.099      0.636      0.525      -0.132       0.258
tipo_residencia[T.Gover

# 2

In [57]:
df2 = smf.ols(''' np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos, Treatment(7)) 
                + tipo_renda + idade + estado_civil + tipo_residencia + idade
                + tempo_emprego + qt_pessoas_residencia''', data = df).fit()
df2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     262.3
Date:                Fri, 26 Jul 2024   Prob (F-statistic):               0.00
Time:                        16:26:09   Log-Likelihood:                -13593.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12400   BIC:                         2.744e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             4.5872      1.026      4.471      0.000       2.576       6.598
sexo[T.M]                             0.7807      0.015     53.218      0.000       0.752       0.809
posse_de_veiculo[T.True]              0.0523      0.014      3.703      0.000       0.025       0.080
posse_de_imovel[T.True]               0.0856      0.014      6.104      0.000       0.058       0.113
C(qtd_filhos, Treatment(7))[T.0]      2.0123      0.822      2.447      0.014       0.400       3.624
C(qtd_filhos, Treatment(7))[T.1]      1.7693      0.725      2.442      0.015       0.349       3.190
C(qtd_filhos, Treatment(7))[T.2]      1.5197      0.630      2.411      0.016       0.284       2.755
C(qtd_filhos, Treatment(7))[T.3]      1.1037      0.542      2.035      0.042       0.040       2.167
C(qtd_filhos, Treatment(7))[T.4]      1.1904      0.490      2.427      0.015       0.229       2.152
C(qtd_filhos, Treatment(7))[T.5]      0.9637      0.645      1.495      0.135      -0.300       2.227
C(qtd_filhos, Treatment(7))[T.14]    -2.7160      0.969     -2.803      0.005      -4.615      -0.817
tipo_renda[T.Bolsista]                0.3018      0.242      1.250      0.211      -0.172       0.775
tipo_renda[T.Empresário]              0.1651      0.015     11.075      0.000       0.136       0.194
tipo_renda[T.Pensionista]            -0.2558      0.242     -1.058      0.290      -0.730       0.218
tipo_renda[T.Servidor público]        0.0730      0.022      3.293      0.001       0.030       0.116
estado_civil[T.Separado]              0.3291      0.112      2.951      0.003       0.110       0.548
estado_civil[T.Solteiro]              0.2694      0.109      2.467      0.014       0.055       0.483
estado_civil[T.União]                -0.0371      0.025     -1.470      0.142      -0.087       0.012
estado_civil[T.Viúvo]                 0.3655      0.116      3.155      0.002       0.138       0.593
tipo_residencia[T.Casa]              -0.0363      0.054     -0.670      0.503      -0.143       0.070
tipo_residencia[T.Com os pais]       -0.0172      0.060     -0.285      0.776      -0.136       0.101
tipo_residencia[T.Comunitário]       -0.0772      0.106     -0.731      0.465      -0.284       0.130
tipo_residencia[T.Estúdio]            0.0816      0.100      0.820      0.413      -0.114       0.277
tipo_residencia[T.Governamental]     -0.0563      0.066     -0.852      0.394      -0.186       0.073
idade                                 0.0052      0.001      6.493      0.000       0.004       0.007
tempo_emprego                         0.0617      0.001     59.215      0.000       0.060       0.064
qt_pessoas_residencia                 0.2890      0.108      2.676      0.007       0.077       0.501
=======================

O modelo declinou no **$R^2$** e o **AIC** aumentou 

# 3

In [73]:
df1 = smf.ols(''' np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel
                  + idade + tempo_emprego''', data = df).fit()
df1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Fri, 26 Jul 2024   Prob (F-statistic):               0.00
Time:                        16:40:03   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    7.2550      0.032    228.981      0.000       7.193       7.317
sexo[T.M]                    0.7700      0.015     52.851      0.000       0.741       0.799
posse_de_veiculo[T.True]     0.0579      0.014      4.116      0.000       0.030       0.085
posse_de_imovel[T.True]      0.0880      0.014      6.379      0.000       0.061       0.115
idade                        0.0044      0.001      5.871      0.000       0.003       0.006
tempo_emprego                0.0611      0.001     59.199      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.536   Jarque-Bera (JB):                1.221
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.019   Cond. No.                         209.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Após uns ajustes parece que o modelo perdeu um pouco da qualidade referente ao $R^2$ comparado ao primeiro modelo